In [40]:
import pandas as pd
df_train=pd.read_csv("Train.csv")
df_test=pd.read_csv("Test.csv")

In [41]:
df_train['source']=df_train.groupby('Category').ngroup()

In [42]:
group=df_train.groupby(['source', 'Category']).size().reset_index()

In [43]:
group=group[['source','Category']]

In [44]:
group

,source,Category
0,0,business
1,1,entertainment
2,2,politics
3,3,sport
4,4,tech


In [45]:
df_train

,ArticleId,Text,Category,source
0,1833,worldcom ex-boss launches defence lawyers defe...,business,0
1,154,german business confidence slides german busin...,business,0
2,1101,bbc poll indicates economic gloom citizens in ...,business,0
3,1976,lifestyle governs mobile choice faster bett...,tech,4
4,917,enron bosses in $168m payout eighteen former e...,business,0
...,...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment,1
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment,1
1487,1590,weak dollar hits reuters revenues at media gro...,business,0
1488,1587,apple ipod family expands market apple has exp...,tech,4


In [46]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
stopWords = set(stopwords.words('english'))

def textcleaner_stem(Text): # Need Change here
    
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', Text.lower()) # Need Change here
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        if w not in stopWords:
            if len(w)>2:
                clean.append(ps.stem(w))
    return ' '.join(clean)

In [47]:
df_train['clean_text_stem'] = df_train.Text.apply(lambda x: textcleaner_stem(x))
df_test['clean_text_stem'] = df_test.Text.apply(lambda x: textcleaner_stem(x))

In [48]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stopWords = set(stopwords.words('english'))

def textcleaner_lemmas(Text): # Need Change here
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', Text.lower()) # Need Change here
    cleaner = word_tokenize(cleaner)
    ps = PorterStemmer()
    clean = []
    for w in cleaner:
        if w not in stopWords:
            if len(w)>2:
                clean.append(lemmatizer.lemmatize(w))
    return ' '.join(clean)

In [49]:
df_train['clean_text_lemma'] = df_train.Text.apply(lambda x: textcleaner_lemmas(x))
df_test['clean_text_lemma'] = df_test.Text.apply(lambda x: textcleaner_lemmas(x))

In [50]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=6, strip_accents='ascii', analyzer='word', lowercase=True,
                             ngram_range=(1,2))

x_train_stem = vectorizer.fit_transform(df_train['clean_text_stem'])
y_train_stem = df_train['source']
x_test_stem = vectorizer.transform(df_test['clean_text_stem'])
#y_test_stem = df_test['source']
features_train = vectorizer.get_feature_names()

In [51]:
from sklearn.naive_bayes import MultinomialNB

#Initialize and fit
nb = MultinomialNB()
nb.fit(x_train_stem, y_train_stem)

# Apply to testing data
y_pred_stem = nb.predict(x_test_stem)


In [52]:
df_test['source']=y_pred_stem

In [55]:
df_test=df_test.drop(['clean_text_lemma', 'clean_text_stem'], axis=1)

In [56]:
df_test

,ArticleId,Text,source
0,1018,qpr keeper day heads for preston queens park r...,3
1,1319,software watching while you work software that...,4
2,1138,d arcy injury adds to ireland woe gordon d arc...,3
3,459,india s reliance family feud heats up the ongo...,0
4,1020,boro suffer morrison injury blow middlesbrough...,3
...,...,...,...
730,1923,eu to probe alitalia state aid the european ...,0
731,373,u2 to play at grammy awards show irish rock ba...,1
732,1704,sport betting rules in spotlight a group of mp...,0
733,206,alfa romeos to get gm engines fiat is to sto...,0


In [57]:
df_test=df_test.merge(group, on='source', how='left')

In [58]:
df_test=df_test.drop(['source'], axis=1)

In [59]:
df_test

,ArticleId,Text,Category
0,1018,qpr keeper day heads for preston queens park r...,sport
1,1319,software watching while you work software that...,tech
2,1138,d arcy injury adds to ireland woe gordon d arc...,sport
3,459,india s reliance family feud heats up the ongo...,business
4,1020,boro suffer morrison injury blow middlesbrough...,sport
...,...,...,...
730,1923,eu to probe alitalia state aid the european ...,business
731,373,u2 to play at grammy awards show irish rock ba...,entertainment
732,1704,sport betting rules in spotlight a group of mp...,business
733,206,alfa romeos to get gm engines fiat is to sto...,business


In [62]:
df_train=df_train.drop(['source','clean_text_stem','clean_text_lemma'], axis=1)

In [63]:
df_train

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [65]:
final=pd.concat([df_train, df_test])